Cambios y consideraciones:
Sin label: Eliminamos el argumento label en to_gate(), ya que qrisp no lo admite.
Visualización: Usamos print(qc) para mostrar el circuito de manera textual, dado que qc.draw("mpl") no es compatible con qrisp.
Difusión y oráculo: Se implementan las mismas operaciones de Grover usando qrisp.

In [ ]:
import qrisp

def grover_oracle_qrisp(n, marked_state):
    
    oracle = qrisp.QuantumCircuit(n)
    for i, bit in enumerate(reversed(marked_state)):
        if bit == '0':
            oracle.x(i)

    oracle.h(n - 1)
    oracle.mcx(list(range(n - 1)), n - 1)  # Control multi-qubit
    oracle.h(n - 1)

    for i, bit in enumerate(reversed(marked_state)):
        if bit == '0':
            oracle.x(i)
    return oracle.to_gate()

def grover_diffusion_qrisp(n):
    
    diffusion = qrisp.QuantumCircuit(n)

    diffusion.h(range(n))
    diffusion.x(range(n))

    diffusion.h(n - 1)
    diffusion.mcx(list(range(n - 1)), n - 1)
    diffusion.h(n - 1)

    diffusion.x(range(n))
    diffusion.h(range(n))

    return diffusion.to_gate()

def grover_algorithm_qrisp(n, marked_state, iterations=1):
  
    qc = qrisp.QuantumCircuit(n, n)
    qc.h(range(n))
    oracle = grover_oracle_qrisp(n, marked_state)
    diffusion = grover_diffusion_qrisp(n)

    # Iterar Grover
    for _ in range(iterations):
        qc.append(oracle, range(n))
        qc.append(diffusion, range(n))
    qc.measure(range(n), range(n))
    return qc

n = 3
marked_state = "101"

# Create algorithm
iterations = int(np.pi / 4 * np.sqrt(2**n))  # Número óptimo de iteraciones
qc = grover_algorithm_qrisp(n, marked_state, iterations)

# Visualizar el circuito
print(qc)


       ┌───┐┌───────────────┐┌───────────────┐┌───────────────┐»
qb_58: ┤ H ├┤0              ├┤0              ├┤0              ├»
       ├───┤│               ││               ││               │»
qb_59: ┤ H ├┤1 circuit14020 ├┤1 circuit14020 ├┤1 circuit14020 ├»
       ├───┤│               ││               ││               │»
qb_60: ┤ H ├┤2              ├┤2              ├┤2              ├»
       └───┘└───────────────┘└───────────────┘└───────────────┘»
 cb_2: ════════════════════════════════════════════════════════»
                                                               »
 cb_3: ════════════════════════════════════════════════════════»
                                                               »
 cb_4: ════════════════════════════════════════════════════════»
                                                               »
«       ┌───────────────┐┌─┐      
«qb_58: ┤0              ├┤M├──────
«       │               │└╥┘┌─┐   
«qb_59: ┤1 circuit14020 ├─╫─┤M├───
«       │      